In [2]:
import cv2
import numpy as np
from scipy import ndimage
import math
import import_ipynb
import copy
import operator
def extract_sudoku(image, model, old_sudoku,flag5,user_grid):

    ratio2 = 3
    kernel_size = 3
    lowThreshold = 30
    jojo = 0
    ratio2 = 3
    kernel_size = 3
    lowThreshold = 30
    verti = []
    horti = []
    Points=[]
    sudoku1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    sudoku1 = cv2.blur(sudoku1, (3,3))
    # Apply Canny edge detection
   
    edges = cv2.Canny(sudoku1, lowThreshold, lowThreshold*ratio2, kernel_size)
    lines = cv2.HoughLines(edges, 2, np.pi/180, 250, 0, 0)
    
    if (lines is not None):
        lines = sorted(lines, key=lambda line:line[0][0])
        # Define the position of horizontal and vertical line
        diff_ngang = 0
        diff_doc = []
        #lines_1=[]
        lines_1 = np.zeros((4, 3), dtype = "float32")
        for i in range(len(lines)):
            for rho,theta in lines[i]:
            #    print('c')
             #   a = np.cos(theta)
             #   b = np.sin(theta)
              #  x0 = a*rho
               # y0 = b*rho
               # x1 = int(x0 + 1000*(-b))
               # y1 = int(y0 + 1000*(a))
                #x2 = int(x0 - 1000*(-b))
               # y2 = int(y0 - 1000*(a))
         # If b > 0.5, the angle must be greater than 45 degree
         # so we consider that line as a vertical line
                if (theta > 1.55 and theta < 1.625):
                
                    if(rho-diff_ngang>25):
                    #print(b)
                        diff_ngang=rho
                        verti.append(lines[i])
                    #lines_1.append([rho,theta, 0])
                if (theta < 0.1 or theta > 3.1):
                #print(theta)
                #if(abs(rho-all(diff_doc)>25):
                    if (all( abs(abs(rho) - abs(x)) > 25 for x in diff_doc)):
                        print(theta)
                        diff_doc.append(rho)
                        horti.append(lines[i])
                    #lines_1.append([rho,theta, 1])
        if horti == []:
            return image,flag5,old_sudoku,user_grid
        if verti == []:
            return image,flag5,old_sudoku,user_grid
        l = abs(horti[0][0][0])
        s = abs(horti[0][0][0])
        for i in range(len(horti)):
            if abs(horti[i][0][0]) <= s:
                s = abs(horti[i][0][0])
                lines_1[0][0] = horti[i][0][0]
                lines_1[0][1] = horti[i][0][1]
                lines_1[0][2] = 1
            if abs(horti[i][0][0]) >= l:
                l = abs(horti[i][0][0])
                lines_1[1][0] = horti[i][0][0]
                lines_1[1][1] = horti[i][0][1]
                lines_1[1][2] = 1
        l1 = verti[0][0][0]
        s1 = verti[0][0][0]
        for i in range(len(verti)):
            if verti[i][0][0] <= s1:
                s1 = verti[i][0][0]
                lines_1[2][0] = verti[i][0][0]
                lines_1[2][1] = verti[i][0][1]
                lines_1[2][2] = 0
            if verti[i][0][0] >= l1:
                l1 = verti[i][0][0]
                lines_1[3][0] = verti[i][0][0]
                lines_1[3][1] = verti[i][0][1]
                lines_1[3][2] = 0 
            
        if len(horti) < 9:
            return image,flag5,old_sudoku,user_grid
        if len(verti) < 9:
            return image,flag5,old_sudoku,user_grid
            
        print('verti')
        print(len(verti))
        print(len(horti))
        for i in range(len(lines_1)):
                if(lines_1[i][2] == 0):
                    for j in range(len(lines_1)):
                        if (lines_1[j][2]==1):
                            theta1=lines_1[i][1]
                            theta2=lines_1[j][1]
                            p1=lines_1[i][0]
                            p2=lines_1[j][0]
                            xy = np.array([[np.cos(theta1), np.sin(theta1)], [np.cos(theta2), np.sin(theta2)]])
                            p = np.array([p1,p2])
                            res = np.linalg.solve(xy, p)
                            Points.append(res)
        
    
    if (len(Points) != 4):
        return image,flag5,old_sudoku,user_grid
    sq = np.zeros((4, 2), dtype = "float32")
    sq[0] = Points[0]
    sq[1] = Points[1]
    sq[2] = Points[3]
    sq[3] = Points[2]
    
    

    
    x = []
    y = []
    for i in range(4):
        x.append(sq[i][0])
        y.append(sq[i][1])
    def s(a,b,c,d):
        l = np.sqrt(((a - b) ** 2) + ((b - d) ** 2))
        return l 
    sideA = s(x[2],x[3],y[2],y[3])
    sideB = s(x[1],x[0],y[1],y[0])
    sideC = s(x[1],x[2],y[1],y[2])
    sideD = s(x[0],x[3],y[0],y[3])
    max_width = max(int(sideA), int(sideB))
    max_height = max(int(sideC), int(sideD))

    
    dst = np.array([
        [0, 0],
        [max_width - 1, 0],
        [max_width - 1, max_height - 1],
        [0, max_height - 1]], dtype = "float32")

    filename1 = r"C:\Users\vvash\cnn\originalvv.jpg"
    cv2.imwrite(filename1, image)
    perspective_m = cv2.getPerspectiveTransform(sq, dst)
    warped_sudoku = cv2.warpPerspective(image, perspective_m, (max_width, max_height))
    filename1 = r"C:\Users\vvash\cnn\warpedvv.jpg"
    cv2.imwrite(filename1, warped_sudoku)
    displayed_warp = np.copy(warped_sudoku)

    if flag5 == 1:
        displayed_warp = write_solution_on_image(displayed_warp, old_sudoku, user_grid)
        print('warpvvv')
    
        
           
        result_sudoku = cv2.warpPerspective(displayed_warp, perspective_m, (image.shape[1], image.shape[0])
                                        , flags=cv2.WARP_INVERSE_MAP)
        result = np.where(result_sudoku.sum(axis=-1,keepdims=True)!=0, result_sudoku, image)
        #filename = r"C:\Users\vvash\cnn\vvv2.jpg"
        #cv2.imwrite(filename, result)
        return result,flag5,old_sudoku,user_grid
    warped_sudoku = cv2.cvtColor(warped_sudoku,cv2.COLOR_BGR2GRAY)
    warped_sudoku = cv2.GaussianBlur(warped_sudoku, (5,5), 0)
    warped_sudoku = cv2.adaptiveThreshold(warped_sudoku, 255, 1, 1, 11, 2)
  #  warp = cv2.bitwise_not(warp)
    _, warped_sudoku = cv2.threshold(warped_sudoku, 150, 255, cv2.THRESH_BINARY)
    filename1 = r"C:\Users\vvash\cnn\wwwvv.jpg"
    cv2.imwrite(filename1, warped_sudoku)
    size = 9
    grid = []
    #for i in range(SIZE):
       # row = []
        #for j in range(SIZE):
        #    row.append(0)
        #grid.append(row)
    for r in range(0, size):
        grid.append([0 for c in range(0, size)])
    height = warped_sudoku.shape[0] // 9
    width = warped_sudoku.shape[1] // 9

    offset_width = math.floor(width / 10)    
    offset_height = math.floor(height / 10)
    for i in range(size):
        for j in range(size):

            
            crop_image = warped_sudoku[height*i+offset_height:height*(i+1)-offset_height, width*j+offset_width:width*(j+1)-offset_width]        
            ratio = 0.6         

           # crop_image = cv2.bitwise_not(crop_image)
            
            crop_image = largest_connected_component(crop_image)
           
               
            
            crop_image = cv2.resize(crop_image, (28,28))
            filename = r"C:\Users\vvash\cnn\v1.jpg"
            cv2.imwrite(filename, crop_image)
           
            if crop_image.sum() >= 28**2*255 - 28 * 1 * 255:
                grid[i][j] == 0
                continue    

            crop_image,f = pre(crop_image,jojo)
            jojo = jojo+1
            if f == 1:
                grid[i][j] = 0
                continue 
            
            filename = r"C:\Users\vvash\cnn\v1.jpg"
            cv2.imwrite(filename, crop_image)
            crop_image = crop_image.reshape(-1, 28, 28, 1)
            crop_image = crop_image.astype('float32')
            crop_image = crop_image / 255
            prediction = model.predict(crop_image) 
            grid[i][j] = np.argmax(prediction[0]) + 1 
            #print(grid[i][j])
            
    print("y")  
    user_grid = copy.deepcopy(grid)


    
   # else:
    if flag == 0:
        co = [True] 
        print('yoooooooooooooo')
        sudoku_helper1(grid,co) 

        if co[0] == False:
            displayed_warp = write_solution_on_image(displayed_warp, grid, user_grid)
            print('warp')
            filename1 = r"C:\Users\vvash\cnn\vvv5.jpg"
            cv2.imwrite(filename1, displayed_warp)
            old_sudoku = copy.deepcopy(grid)      

        
            flag5 = flag5 + 1
            result_sudoku = cv2.warpPerspective(displayed_warp, perspective_m, (image.shape[1], image.shape[0])
                                        , flags=cv2.WARP_INVERSE_MAP)
            result = np.where(result_sudoku.sum(axis=-1,keepdims=True)!=0, result_sudoku, image)
            filename = r"C:\Users\vvash\cnn\vvv2.jpg"
            cv2.imwrite(filename, result)
            return result,flag5,old_sudoku,user_grid
   
    
    result_sudoku = cv2.warpPerspective(displayed_warp, perspective_m, (image.shape[1], image.shape[0])
                                        , flags=cv2.WARP_INVERSE_MAP)
    result = np.where(result_sudoku.sum(axis=-1,keepdims=True)!=0, result_sudoku, image)
    return result,flag5,old_sudoku,user_grid


def write_solution_on_image(image, grid, user_grid):
    SIZE = 9
    width = image.shape[1] // 9
    height = image.shape[0] // 9
    for i in range(SIZE):
        for j in range(SIZE):
            if(user_grid[i][j] != 0):   
                continue                
            text = str(grid[i][j])
            off_set_x = width // 15
            off_set_y = height // 15
            font = cv2.FONT_HERSHEY_SIMPLEX
            (text_height, text_width), baseLine = cv2.getTextSize(text, font, fontScale=1, thickness=3)
            marginX = math.floor(width / 7)
            marginY = math.floor(height / 7)
            font_scale = 0.6 * min(width, height) / max(text_height, text_width)
            text_height *= font_scale
            text_width *= font_scale
            bottom_left_corner_x = width*j + math.floor((width - text_width) / 2) + off_set_x
            bottom_left_corner_y = height*(i+1) - math.floor((height - text_height) / 2) + off_set_y
            image = cv2.putText(image, text, (bottom_left_corner_x, bottom_left_corner_y), 
                                                  font, font_scale, (255,0,0), thickness=3, lineType=cv2.LINE_AA)
            
    return image


def findCorners(img):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    polygon = contours[0]

    bottom_right, _ = max(enumerate([pt[0][0] + pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
    top_left, _ = min(enumerate([pt[0][0] + pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
    bottom_left, _ = min(enumerate([pt[0][0] - pt[0][1] for pt in polygon]), key=operator.itemgetter(1))

    top_right, _ = max(enumerate([pt[0][0] - pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
    return [polygon[top_left][0], polygon[top_right][0], polygon[bottom_right][0], polygon[bottom_left][0]]
def largest_connected_component(image):

    image = image.astype('uint8')
    n, output, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=8)
    if n <= 1:
        new_img = np.zeros(image.shape)
        new_img.fill(255)
        return new_img

    pos = 1
    sizes = stats[:, -1]   
    max_size = sizes[1]    # sizes[0] will have the size of the background 

    for i in range(2, n):
        if sizes[i] > max_size:
            pos = i
            max_size = sizes[i]

    img2 = np.zeros(output.shape)
    img2.fill(255)
    img2[output == pos] = 0
   # print(stats)
   # print('yyyyyyyyyy')
    return img2



def get_best_shift(img):
    cy, cx = ndimage.measurements.center_of_mass(img)
    rows, cols = img.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)
    return shiftx, shifty


def shift(img,sx,sy):
    rows,cols = img.shape
    M = np.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

def pre(crop_image,j):
            
            filename = r"C:\Users\vvash\cnn\d"
            filename = filename + str(j)  + 'cppppp.jpg'
            cv2.imwrite(filename, crop_image)
            print('saved')
            flag2=0
         #   center_width = crop_image.shape[1] // 2
         #   center_height = crop_image.shape[0] // 2
            c_w = 28 // 2
            c_h = 28 // 2
            x_start = c_h // 2
            x_end = c_h // 2 + c_h
            y_start = c_w // 2
            y_end = c_w // 2 + c_w
            center_region = crop_image[x_start:x_end, y_start:y_end]
            if center_region.sum() >= c_w * c_h * 255 - 255:
                flag2 = 1 
                print('less')
            else:
            
                rows, cols = crop_image.shape

           
                _, crop_image = cv2.threshold(crop_image, 200, 255, cv2.THRESH_BINARY) 
                crop_image = crop_image.astype(np.uint8)

            
                crop_image = cv2.bitwise_not(crop_image)   
                cy, cx = ndimage.measurements.center_of_mass(crop_image)
                rows, cols = crop_image.shape
                shiftx = np.round(cols/2.0-cx).astype(int)
                shifty = np.round(rows/2.0-cy).astype(int)
                #shift_x, shift_y = get_best_shift(crop_image)
                M = np.float32([[1,0,shiftx],[0,1,shifty]])
                crop_image = cv2.warpAffine(crop_image,M,(cols,rows))
                #shifted = shift(crop_image,shift_x,shift_y)
                #crop_image = shifted
                filename = r"C:\Users\vvash\cnn\d"
                filename = filename + str(j)  + 'dppppp.jpg'
                cv2.imwrite(filename, crop_image)
                print('saved')
            return crop_image,flag2

def check_square(sides,points):
    cs = 0
    def calc_angle(s1,s2): 
        v1 = s1 / np.linalg.norm(s1)
        v2 = s2 / np.linalg.norm(s2)
        dot_droduct = np.dot(v1, v2)
        angle = np.arccos(dot_droduct)
        angle = angle * 57.2958 
        return angle
    angle1 = calc_angle(sides[0],sides[3])
    if  not (abs(90 - angle1) < 20):
        cs = cs + 1
        return cs
    angle2 = calc_angle(sides[0],sides[1])
    if  not (abs(90 - angle2) < 20):
        cs = cs + 1
        return cs
    angle3 = calc_angle(sides[1],sides[2])
    if  not (abs(90 - angle3) < 20):
        cs = cs + 1
        return cs
    angle4 = calc_angle(sides[3],sides[2])
    if  not (abs(90 - angle4) < 20):
        cs = cs + 1
        return cs

    
    
    p1 = math.sqrt((points[0][0]-points[1][0])**2 + (points[0][1]-points[1][1])**2)
    p2 = math.sqrt((points[0][0]-points[3][0])**2 + (points[0][1]-points[3][1])**2)
    p3 = math.sqrt((points[1][0]-points[2][0])**2 + (points[1][1]-points[2][1])**2)
    p4 = math.sqrt((points[2][0]-points[3][0])**2 + (points[2][1]-points[3][1])**2)
    if max(p1, p2, p3, p4) > 1.2 * min(p1, p2, p3, p4):
        cs = cs + 1
        return cs
    return cs